In [10]:
import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 12

In [2]:
import pytfa
from optlang.exceptions import SolverError

#from cobra.core.model import SolverNotFound
from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data
from pytfa.optim.relaxation import relax_dgo
import pickle
from pytfa.optim.variables import LogConcentration
#thermo_database = '/projectnb2/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/data/thermo_data.thermodb'
#root_dir = '/projectnb/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/tests/singleCoenzymeModel.08272021.v2'
thermo_database = '../assets/thermo_data.thermodb'
model_path = '../assets'

CPLEX = 'optlang-cplex'
GUROBI = 'optlang-gurobi'
GLPK = 'optlang-glpk'
solver = GUROBI

# Load reaction DB
print("Loading thermo data...")
thermo_data = load_thermoDB(thermo_database)

print("Done !")
#biomass_rxn = 'BIOMASS_Ec_iJO1366_WT_53p95M'
biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'


model_id = 'iJO1366.json'

# We import pre-compiled data as it is faster for bigger models
cobra_model = load_json_model(model_path + '/' + model_id)

lexicon = read_lexicon(model_path + '/iJO1366/lexicon.csv')
compartment_data = read_compartment_data(model_path + '/iJO1366/compartment_data.json')

# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

# Annotate the cobra_model
annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)


mytfa.name = 'iJO1366'
mytfa.solver = solver
mytfa.objective = biomass_rxn


Loading thermo data...
Done !
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpqoqr0bw7.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2021-08-29 17:11:59,181 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [3]:
def single_coenzyme_transform(model,reaction_id):
    
    met = {}
    met['nad'] = np.where([x.id == 'nad_c' for x in model.metabolites])[0][0]
    met['nadp'] = np.where([x.id == 'nadp_c' for x in model.metabolites])[0][0]
    met['nadh'] = np.where([x.id == 'nadh_c' for x in model.metabolites])[0][0]
    met['nadph'] = np.where([x.id == 'nadph_c' for x in model.metabolites])[0][0]

    met_objs = {}
    met_objs['nad'] = [x for x in model.metabolites if x.id == 'nad_c'][0]
    met_objs['nadh'] = [x for x in model.metabolites if x.id == 'nadh_c'][0]
    met_objs['nadp'] = [x for x in model.metabolites if x.id == 'nadp_c'][0]
    met_objs['nadph'] = [x for x in model.metabolites if x.id == 'nadph_c'][0]
    
    rxn = [x for x in model.reactions if x.id == reaction_id][0].copy()
    # make a new dictionary with coenzyme swapped
    v = {x:y for x,y in rxn.metabolites.items() if x.id in [x + '_c' for x in list(met)]}
    
    nad_stoich = 0;
    nadh_stoich  = 0;
    set([x.id for x in m.reactions.get_by_id(rxnid).metabolites])
    if len(v) > 1:
        v2 = {}
        for x,y in v.items():
            if x.id == 'nadph_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadh']] = y
            elif x.id == 'nadp_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nad']] = y
            elif x.id == 'nadh_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadph']] = y
            elif x.id == 'nad_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nadp']] = y

        v2[met_objs['nad']] = nad_stoich
        v2[met_objs['nadh']] = nadh_stoich

        rxn.subtract_metabolites(v)
        rxn.add_metabolites(v2)
        #rxn.id = rxn.id + '[condensed]'
        # keep old reaction ID
        #print('adding new reaction '+ rxn.id)
        model.remove_reactions([x for x in model.reactions if x.id == reaction_id][0])
        model.add_reaction(rxn)
        coverted = True
        return model,coverted
    else:
        coverted = False
        return model,coverted

In [8]:
#cobra_model.summary()

In [11]:
m  = cobra_model.copy()

rxns_to_remove = ['NADTRHD','NADPPPS','NADK']
rxn_ids = [x.id for x in cobra_model.reactions]

#m.remove_reactions([x for x in m.reactions if x.id == 'NADTRHD'][0])
#m.remove_reactions([x for x in m.reactions if x.id == 'NADPPPS'][0])
#m.remove_reactions([x for x in m.reactions if x.id == 'NADK'][0])
#m.remove_reactions([x for x in m.reactions if x.id in rxns_to_remove][0])

m.remove_reactions(rxns_to_remove)

rxn_ids = [x for x in rxn_ids if x not in rxns_to_remove]
#rxn_ids = [x for x in rxn_ids if x not in ['Ec_biomass_iJO1366_WT_53p95M','Ec_biomass_iJO1366_core_53p95M']]

converted_rxns = []
for rxnid in rxn_ids:
    m,conv = single_coenzyme_transform(m,rxnid)
    if conv:
        converted_rxns.append(rxnid)

        
#m.remove_reactions([x for x in m.reactions if x.id == 'NADTRHD[condensed]'][0])
#m.remove_reactions([x for x in m.reactions if x.id == 'NADPPPS[condensed]'][0])
#m.remove_reactions([x for x in m.reactions if x.id == 'NADK[condensed]'][0])
#m.remove_reactions([x for x in m.reactions if x.id == 'NADDP'][0])

m.id = 'iJO1366[NAD]'
#m.objective = 'BIOMASS_Ec_iJO1366_WT_53p95M'
m.objective = 'Ec_biomass_iJO1366_WT_53p95M'

met_objs = {}
met_objs['nad'] = [x for x in m.metabolites if x.id == 'nad_c'][0]
met_objs['nadh'] = [x for x in m.metabolites if x.id == 'nadh_c'][0]
met_objs['nadp'] = [x for x in m.metabolites if x.id == 'nadp_c'][0]
met_objs['nadph'] = [x for x in m.metabolites if x.id == 'nadph_c'][0]
#m.remove_metabolites([met_objs['nadp'],met_objs['nadph']])
m,unusedmets = cobra.manipulation.delete.prune_unused_metabolites(m)

Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp4fxaduj7.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


/Users/joshuagoldford/opt/miniconda3/envs/coenzymes/lib/python3.8/site-packages/cobra/core/model.py:745: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpnb5icx1v.lp
Reading time = 0.02 seconds
: 1807 rows, 5164 columns, 20294 nonzeros


In [16]:
def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)


## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
print('FBA Solution found : {0:.5g}'.format(fba_value))

# fva = flux_variability_analysis(mytfa)

## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

## Info on the cobra_model
mytfa.print_info()

2021-08-29 17:19:00,844 - thermomodel_None - INFO - # Model preparation starting...


FBA Solution found : 0.814


2021-08-29 17:19:03,030 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-29 17:19:03,031 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,032 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,032 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-29 17:19:03,034 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,035 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,218 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-29 17:19:03,219 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,220 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,220 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-29 17:19:03,221 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,222 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-29 17:19:03,342 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-29 17:19:03,343 - thermomodel_None - WARNING - Warning :

                   value
key                     
name             iJO1366
description      iJO1366
num constraints    15034
num variables      15538
num metabolites     1807
num reactions       2585
                             value
key                               
num metabolites(thermo)       1550
num reactions(thermo)         1824
pct metabolites(thermo)  85.777532
pct reactions(thermo)    70.560928


In [18]:
computed_rxns = [x.id for x in mytfa.reactions if x.thermo['computed']]

In [26]:
len(set(converted_rxns).intersection(set(computed_rxns))) / len(converted_rxns)

0.7230046948356808

In [44]:
set(converted_rxns) - (set(computed_rxns))

{'3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR140',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAR40',
 '3OAR60',
 '3OAR80',
 'BSORx',
 'BSORy',
 'CDGR',
 'EAR100x',
 'EAR100y',
 'EAR120x',
 'EAR120y',
 'EAR121x',
 'EAR121y',
 'EAR140x',
 'EAR140y',
 'EAR141x',
 'EAR141y',
 'EAR160x',
 'EAR160y',
 'EAR161x',
 'EAR161y',
 'EAR180x',
 'EAR180y',
 'EAR181x',
 'EAR181y',
 'EAR40x',
 'EAR40y',
 'EAR60x',
 'EAR60y',
 'EAR80x',
 'EAR80y',
 'EGMEACPR',
 'EPMEACPR',
 'Ec_biomass_iJO1366_WT_53p95M',
 'Ec_biomass_iJO1366_core_53p95M',
 'FLDR2',
 'GLUTRR',
 'LIPOS',
 'MOADSUx',
 'NHFRBO',
 'NODOx',
 'NODOy',
 'OGMEACPR',
 'OPMEACPR',
 'OXCOAHDH',
 'PACCOAE',
 'POAACR',
 'SELGTHR2',
 'SELGTHR3',
 'SHCHD2',
 'THZPSN3'}

In [38]:
rxns = [x for x in converted_rxns if x not in ['Ec_biomass_iJO1366_WT_53p95M','Ec_biomass_iJO1366_core_53p95M']]

wt_model = cobra_model.copy()
nad_only_model = m.copy()

for r in rxns:
    if r in computed_rxns:
        nad_only_model.reactions.get_by_id(r).lower_bound = -1000
        nad_only_model.reactions.get_by_id(r).upper_bound = 1000
        wt_model.reactions.get_by_id(r).lower_bound = -1000
        wt_model.reactions.get_by_id(r).upper_bound = 1000


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpo_sxt2gx.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpzrvpe_zt.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20294 nonzeros


In [42]:
nad_only_model.slim_optimize() / wt_model.slim_optimize()

1.0000013385537698

In [43]:
#m_unconstrained.summary()
cobra.io.json.save_json_model(nad_only_model,'../assets/iJO1366_NAD_semi-unconstrained.json')
cobra.io.json.save_json_model(wt_model,'../assets/iJO1366_WT_semi-unconstrained.json')

In [1]:
r

NameError: name 'r' is not defined